# 5 Busca de hiperparâmetros

In [3]:
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [4]:
import os
print("Diretório Atual:", os.getcwd())

Diretório Atual: C:\Users\educl


In [5]:
# Importando o Dataset
file = open('A652.pickle' , 'rb')
(X_train , y_train , X_val , y_val , X_test , y_test) = pickle.load(file)
print(f"Shapes: ", X_train.shape , X_val.shape , X_test.shape, y_train.shape , y_val.shape , y_test.shape)

Shapes:  (10012, 24) (2506, 24) (9582, 24) (10012, 1) (2506, 1) (9582, 1)


In [6]:
#Substituindo todos os valores diferentes de zero por 1

y_train[y_train != 0] = 1
y_val[y_val != 0] = 1
y_test[y_test != 0] = 1

# Salve as variáveis
with open('A652_substituido.pickle', 'wb') as file:
  pickle.dump((y_train, y_val, y_test), file)

In [7]:
print(f"Shapes: ", y_train.shape , y_val.shape , y_test.shape)

Shapes:  (10012, 1) (2506, 1) (9582, 1)


#### Concatenar conjuntos de treinamento e validação

In [8]:
X_train_combined = np.concatenate((X_train, X_val), axis=0)
y_train_combined = np.concatenate((y_train, y_val), axis=0)

In [9]:
print("X_train_combined:", X_train_combined.shape)
print("y_train_combined:", y_train_combined.shape)

X_train_combined: (12518, 24)
y_train_combined: (12518, 1)


In [10]:
Ad = DecisionTreeClassifier()
Ad.fit(X_train_combined, y_train_combined)

DecisionTreeClassifier()

In [11]:
y_pred_Ad = Ad.predict(X_test)

# Exibir métricas para o modelo padrão
print("Métricas para o modelo padrão (DecisionTreeClassifier):")
print("Acurácia:", accuracy_score(y_test, y_pred_Ad))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_Ad))
print("\nMatriz de Confusão:")
print(confusion_matrix(y_test, y_pred_Ad))

Métricas para o modelo padrão (DecisionTreeClassifier):
Acurácia: 0.899603423084951

Classification Report:
              precision    recall  f1-score   support

         0.0       0.94      0.95      0.95      8769
         1.0       0.40      0.35      0.37       813

    accuracy                           0.90      9582
   macro avg       0.67      0.65      0.66      9582
weighted avg       0.89      0.90      0.90      9582


Matriz de Confusão:
[[8335  434]
 [ 528  285]]


In [12]:
# Modelo otimizado
model_rs = DecisionTreeClassifier()

In [13]:
# Buscando os hiperparâmetros a serem ajustados e suas respectivas faixas

param_dist = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [None, 'sqrt', 'log2', 0.2, 0.5, 0.7, 0.8, 0.9]
}

O Random Search seleciona combinações aleatórias de valores de hiperparâmetros (dentro de intervalos predefinidos para cada hiperparâmetro) para treinar e avaliar os modelos candidatos. Em vez de testar todas as combinações possíveis de valores, a busca aleatória realiza um número pré-definido de iterações, testando uma combinação aleatória de hiperparâmetros em cada iteração. Neste exemplo, usaremos 50 iterações e um CV = 10 O parâmetro indica o número de dobras (folds) utilizadas na validação cruzada. 

In [14]:
# Criar um objeto RandomizedSearchCV para pesquisa aleatória
random_search = RandomizedSearchCV(
    model_rs,
    param_distributions=param_dist,
    n_iter=50,
    scoring='accuracy',
    cv=10,
    n_jobs=-1
)

In [15]:
random_search.fit(X_train_combined, y_train_combined)

RandomizedSearchCV(cv=10, estimator=DecisionTreeClassifier(), n_iter=50,
                   n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [None, 10, 20, 30, 40, 50],
                                        'max_features': [None, 'sqrt', 'log2',
                                                         0.2, 0.5, 0.7, 0.8,
                                                         0.9],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'splitter': ['best', 'random']},
                   scoring='accuracy')

In [16]:
y_pred_rs = random_search.predict(X_test)

In [17]:
# Exibir os melhores hiperparâmetros encontrados

print("\nMelhores hiperparâmetros encontrados:")
print(random_search.best_params_)


Melhores hiperparâmetros encontrados:
{'splitter': 'random', 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 0.7, 'max_depth': 10, 'criterion': 'gini'}


In [18]:
# Avaliar o modelo otimizado no conjunto de teste

y_pred_rs = random_search.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_rs)
print("Métricas para o modelo Otimizado (DecisionTreeClassifier):")
print(f'\nAcurácia no conjunto de teste usando os melhores parâmetros: {accuracy}')


Métricas para o modelo Otimizado (DecisionTreeClassifier):

Acurácia no conjunto de teste usando os melhores parâmetros: 0.9298685034439574


In [87]:
# Exibir as métricas de avaliação

print("\nClassification Report:")
print(classification_report(y_test, y_pred_rs))
print("\nMatriz de Confusão:")
print(confusion_matrix(y_test, y_pred_rs))


Classification Report:
              precision    recall  f1-score   support

         0.0       0.94      0.99      0.96      8769
         1.0       0.76      0.33      0.46       813

    accuracy                           0.93      9582
   macro avg       0.85      0.66      0.71      9582
weighted avg       0.93      0.93      0.92      9582


Matriz de Confusão:
[[8682   87]
 [ 543  270]]
